To remove all component details within the "Content" column in such a way that we only have text. 
1. Drop rows that contain anything different text. (Pauses, Actions, Unintelligble Speech)
1. Remove punctuation marks from every row. (Removing ".", ",", "'", "?", etc.)
1. Always keep the corresponding "StTime" and "EnTime".
1. You can ommit/remove the "Spkr" "Line" or line columns.

In [16]:
import pandas as pd
import numpy as np
import torchaudio
import torch

In [7]:
transcript_dir = "../notebooks/coraal/transcript/text"

transcript_df = pd.read_csv(transcript_dir+"/ATL_se0_ag1_f_01_1.txt", delimiter="\t")
transcript_df.head()

,Line,Spkr,StTime,Content,EnTime
0,1,ATL_se0_ag1_f_01,0.4436,"They talking about, don't send him to his daddy.",2.4068
1,2,ATL_se0_ag1_f_01,2.4068,(pause 0.28),2.6829
2,3,ATL_se0_ag1_f_01,2.6829,You just need to go file for child support.,4.9538
3,4,ATL_int_01,5.1142,[/Oh man/.],5.6125
4,5,ATL_se0_ag1_f_01,5.1488,[Bye.],5.5995


In [10]:
# Iterate through each row in our dataframe
for row in transcript_df.iterrows():
    print(row)
    break

(0, Line                                                      1
Spkr                                       ATL_se0_ag1_f_01
StTime                                               0.4436
Content    They talking about, don't send him to his daddy.
EnTime                                               2.4068
Name: 0, dtype: object)


In [15]:
# Iterates through columns
for data in transcript_df:
    print(data)
    break

Line


In [14]:
for tuples in transcript_df.itertuples():
    display(tuples)
    break

Pandas(Index=0, Line=1, Spkr='ATL_se0_ag1_f_01', StTime=0.4436, Content="They talking about, don't send him to his daddy.", EnTime=2.4068)